In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import tqdm
tqdm.tqdm.pandas()

#MySQL connections
import mysql.connector
from sqlalchemy import create_engine
import sqlalchemy
from sqlalchemy import *
from sqlalchemy.sql import select
from sqlalchemy.orm import Session, sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy_utils import database_exists, create_database, drop_database
from MySQLconfig import username, password

# Import files & Display dataframes

In [3]:
#Import all needed datasets
flightscsv = "flights.csv"
airlines="airline_names.csv"
origin_airports="origin_airport_names.csv"
destination_airports="destination_airport_names.csv"

In [4]:
#big file. includes the all the flights
df = pd.read_csv(flightscsv, low_memory=False)


In [5]:
#file shows the airline long names from https://openflights.org/data.html
airline_names=pd.read_csv(airlines) 

In [6]:
#file shows the airport long names and location (latitude and longitude) from https://openflights.org/data.html
origin_airport_names=pd.read_csv(origin_airports,encoding = 'unicode_escape')
destination_airport_names=pd.read_csv(destination_airports,encoding = 'unicode_escape')

In [7]:
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
airline_names.head()

,AIRLINE,AIRLINE_NAME
0,GNL,135 Airways
1,SEC,3D Aviation
2,Q5,40-Mile Air
3,AQ,Aloha Airlines
4,AAK,Alaska Island Air


In [9]:
origin_airport_names.head()

,ORIGIN_AIRPORT,ORIGIN_AIRPORT_NAME,ORIGIN_AIRPORT_CITY,ORIGIN_LAT,ORIGIN_LONG,TOP30
0,BTI,Barter Island LRRS Airport,Barter Island,70.134003,-143.582001,NO
1,LUR,Cape Lisburne LRRS Airport,Cape Lisburne,68.875099,-166.110001,NO
2,PIZ,Point Lay LRRS Airport,Point Lay,69.732903,-163.005005,NO
3,ITO,Hilo International Airport,Hilo,19.721399,-155.048004,NO
4,ORL,Orlando Executive Airport,Orlando,28.545500,-81.332901,NO


In [10]:
destination_airport_names.head()

,DESTINATION_AIRPORT,DESTINATION_AIRPORT_NAME,DESTINATION_AIRPORT_CITY,DESTINATION_LAT,DESTINATION_LONG,TOP30
0,BTI,Barter Island LRRS Airport,Barter Island,70.134003,-143.582001,NO
1,LUR,Cape Lisburne LRRS Airport,Cape Lisburne,68.875099,-166.110001,NO
2,PIZ,Point Lay LRRS Airport,Point Lay,69.732903,-163.005005,NO
3,ITO,Hilo International Airport,Hilo,19.721399,-155.048004,NO
4,ORL,Orlando Executive Airport,Orlando,28.545500,-81.332901,NO


# File review and manupilation

In [11]:
#Replace NaNs with zero for specific columns
df.update(df[['AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY']].fillna(0))

df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0


In [12]:
#Update CANCELLED and DIVERTED column's row values

#Convert values to string
df.CANCELLED=df.CANCELLED.astype(str)
df.DIVERTED=df.DIVERTED.astype(str)
#df.CANCELLATION_REASON=df.CANCELLATION_REASON.astype(str)

#Replace 0 with Not Cancelled and replace 1 with Cancelled
df['CANCELLED']=df['CANCELLED'].replace(['0'],'Not Cancelled')
df['CANCELLED']=df['CANCELLED'].replace(['1'],'Cancelled')

df['DIVERTED']=df['DIVERTED'].replace(['0'],'Not Diverted')
df['DIVERTED']=df['DIVERTED'].replace(['1'],'Diverted')

df['CANCELLATION_REASON']=df['CANCELLATION_REASON'].fillna('Not Cancelled')


df.CANCELLATION_REASON.unique()
#df.head()

array(['Not Cancelled', 'A', 'B', 'C', 'D'], dtype=object)

In [13]:
#Update DAY_OF_WEEK column's row values

#Convert values to string
df.DAY_OF_WEEK=df.DAY_OF_WEEK.astype(str)

#Replace 0 with Not Cancelled and replace 1 with Cancelled
df['DAY_OF_WEEK']=df['DAY_OF_WEEK'].replace(['1'],'MON')
df['DAY_OF_WEEK']=df['DAY_OF_WEEK'].replace(['2'],'TUE')
df['DAY_OF_WEEK']=df['DAY_OF_WEEK'].replace(['3'],'WED')
df['DAY_OF_WEEK']=df['DAY_OF_WEEK'].replace(['4'],'THU')
df['DAY_OF_WEEK']=df['DAY_OF_WEEK'].replace(['5'],'FRI')
df['DAY_OF_WEEK']=df['DAY_OF_WEEK'].replace(['6'],'SAT')
df['DAY_OF_WEEK']=df['DAY_OF_WEEK'].replace(['7'],'SUN')

df.head(10)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,THU,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0
1,2015,1,1,THU,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0
2,2015,1,1,THU,US,840,N171US,SFO,CLT,20,...,811.0,5.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0
3,2015,1,1,THU,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0
4,2015,1,1,THU,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0
5,2015,1,1,THU,DL,806,N3730B,SFO,MSP,25,...,610.0,8.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0
6,2015,1,1,THU,NK,612,N635NK,LAS,MSP,25,...,509.0,-17.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0
7,2015,1,1,THU,US,2013,N584UW,LAX,CLT,30,...,753.0,-10.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0
8,2015,1,1,THU,AA,1112,N3LAAA,SFO,DFW,30,...,532.0,-13.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0
9,2015,1,1,THU,DL,1173,N826DN,LAS,ATL,30,...,656.0,-15.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0


In [14]:
df['MONTH'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

# Merging of datasets

In [15]:
df1 = pd.merge(df, airline_names, on="AIRLINE", how='inner')
df1.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,AIRLINE_NAME
0,2015,1,1,THU,AA,2336,N3KUAA,LAX,PBI,10,...,-9.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0,American Airlines
1,2015,1,1,THU,AA,258,N3HYAA,LAX,MIA,20,...,-9.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0,American Airlines
2,2015,1,1,THU,AA,1112,N3LAAA,SFO,DFW,30,...,-13.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0,American Airlines
3,2015,1,1,THU,AA,1674,N853AA,LAS,MIA,35,...,-10.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0,American Airlines
4,2015,1,1,THU,AA,371,N3GXAA,SEA,MIA,100,...,1.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0,American Airlines


In [16]:
df1['MONTH'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

In [17]:
df1.loc[df1['MONTH'] == 10]

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,AIRLINE_NAME
498261,2015,10,1,THU,AA,1230,N3DBAA,14747,11298,5,...,-19.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0,American Airlines
498262,2015,10,1,THU,AA,260,N3GNAA,12892,13303,10,...,7.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0,American Airlines
498263,2015,10,1,THU,AA,1982,N914UY,14771,11057,10,...,-11.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0,American Airlines
498264,2015,10,1,THU,AA,448,N179UW,14869,11057,15,...,-5.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0,American Airlines
498265,2015,10,1,THU,AA,2406,N859AA,12892,11298,15,...,-1.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0,American Airlines
498266,2015,10,1,THU,AA,1868,N672AW,12892,10693,35,...,-7.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0,American Airlines
498267,2015,10,1,THU,AA,1965,N544UW,12892,11057,40,...,44.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,41.0,3.0,0.0,American Airlines
498268,2015,10,1,THU,AA,1746,N587UW,14107,11298,50,...,-23.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0,American Airlines
498269,2015,10,1,THU,AA,1051,N865AA,14771,11298,55,...,3.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0,American Airlines
498270,2015,10,1,THU,AA,1113,N3ADAA,12892,13930,59,...,-10.0,Not Diverted,Not Cancelled,Not Cancelled,0.0,0.0,0.0,0.0,0.0,American Airlines


In [18]:
df2 = pd.merge(df1, origin_airport_names, on="ORIGIN_AIRPORT",how='inner')
df2.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,AIRLINE_NAME,ORIGIN_AIRPORT_NAME,ORIGIN_AIRPORT_CITY,ORIGIN_LAT,ORIGIN_LONG,TOP30
0,2015,1,1,THU,AA,2336,N3KUAA,LAX,PBI,10,...,0.0,0.0,0.0,0.0,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30
1,2015,1,1,THU,AA,258,N3HYAA,LAX,MIA,20,...,0.0,0.0,0.0,0.0,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30
2,2015,1,1,THU,AA,115,N3CTAA,LAX,MIA,105,...,0.0,0.0,0.0,0.0,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30
3,2015,1,1,THU,AA,2410,N3BAAA,LAX,DFW,550,...,0.0,0.0,0.0,0.0,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30
4,2015,1,1,THU,AA,1515,N3HMAA,LAX,ORD,600,...,0.0,0.0,0.0,0.0,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30


In [19]:
df2['MONTH'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 10], dtype=int64)

In [20]:
flights_merged = pd.merge(df2, destination_airport_names, on="DESTINATION_AIRPORT",how='inner')
flights_merged.head(10)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ORIGIN_AIRPORT_NAME,ORIGIN_AIRPORT_CITY,ORIGIN_LAT,ORIGIN_LONG,TOP30_x,DESTINATION_AIRPORT_NAME,DESTINATION_AIRPORT_CITY,DESTINATION_LAT,DESTINATION_LONG,TOP30_y
0,2015,1,1,THU,AA,2336,N3KUAA,LAX,PBI,10,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,NO
1,2015,1,2,FRI,AA,2336,N3HPAA,LAX,PBI,10,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,NO
2,2015,1,4,SUN,AA,2336,N3ERAA,LAX,PBI,10,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,NO
3,2015,1,5,MON,AA,2336,N3JHAA,LAX,PBI,10,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,NO
4,2015,1,8,THU,AA,2336,N3FKAA,LAX,PBI,1410,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,NO
5,2015,1,9,FRI,AA,2336,N3DUAA,LAX,PBI,1410,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,NO
6,2015,1,10,SAT,AA,2336,N3GPAA,LAX,PBI,1410,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,NO
7,2015,1,11,SUN,AA,2336,N3LXAA,LAX,PBI,1410,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,NO
8,2015,1,15,THU,AA,2336,N3EVAA,LAX,PBI,1410,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,NO
9,2015,1,16,FRI,AA,2336,N3ERAA,LAX,PBI,1410,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,NO


In [21]:
flights_merged['ORIGIN_AIRPORT_NAME'].unique()

array(['Los Angeles International Airport',
       'Denver International Airport',
       'Dallas Fort Worth International Airport',
       'General Edward Lawrence Logan International Airport',
       'George Bush Intercontinental Houston Airport',
       'John F Kennedy International Airport',
       'Newark Liberty International Airport',
       'Miami International Airport',
       'Ronald Reagan Washington National Airport',
       'Baltimore/Washington International Thurgood Marshall Airport',
       'La Guardia Airport', 'Detroit Metropolitan Wayne County Airport',
       'Washington Dulles International Airport',
       'Charlotte Douglas International Airport',
       'Philadelphia International Airport',
       'Hartsfield Jackson Atlanta International Airport',
       'Pittsburgh International Airport',
       'Bradley International Airport',
       'Chicago OHare International Airport',
       'Theodore Francis Green State Airport',
       'Chicago Midway International Airp

# Top Airports by Origin

In [22]:
#Find out the busiest "n" airports by count of origin flights count. Change the number in the head() function.
busy_airports = flights_merged[["ORIGIN_AIRPORT_NAME"]]

busy_airports['ORIGIN_AIRPORT_NAME'].value_counts().head(30)

Hartsfield Jackson Atlanta International Airport                357862
Chicago OHare International Airport                             295062
Dallas Fort Worth International Airport                         246641
Denver International Airport                                    203903
Los Angeles International Airport                               197838
San Francisco International Airport                             152193
Phoenix Sky Harbor International Airport                        152127
George Bush Intercontinental Houston Airport                    151737
McCarran International Airport                                  137073
Minneapolis-St Paul International/Wold-Chamberlain Airport      117167
Detroit Metropolitan Wayne County Airport                       113985
General Edward Lawrence Logan International Airport             113285
Orlando International Airport                                   112262
La Guardia Airport                                              106337
Charlo

In [23]:
#Dataframe filtered by the busiest airports. See busy_airports df for filtered count
top_airports = flights_merged[flights_merged["TOP30_x"].isin(["top30"])]

top_airports.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ORIGIN_AIRPORT_NAME,ORIGIN_AIRPORT_CITY,ORIGIN_LAT,ORIGIN_LONG,TOP30_x,DESTINATION_AIRPORT_NAME,DESTINATION_AIRPORT_CITY,DESTINATION_LAT,DESTINATION_LONG,TOP30_y
0,2015,1,1,THU,AA,2336,N3KUAA,LAX,PBI,10,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,NO
1,2015,1,2,FRI,AA,2336,N3HPAA,LAX,PBI,10,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,NO
2,2015,1,4,SUN,AA,2336,N3ERAA,LAX,PBI,10,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,NO
3,2015,1,5,MON,AA,2336,N3JHAA,LAX,PBI,10,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,NO
4,2015,1,8,THU,AA,2336,N3FKAA,LAX,PBI,1410,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,top30,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,NO


In [24]:
top_airports['MONTH'].unique()

array([ 1,  2,  3,  4,  5, 12,  6,  7,  8,  9, 11, 10], dtype=int64)

In [25]:
top_airports['ORIGIN_AIRPORT_NAME'].value_counts().head(30)

Hartsfield Jackson Atlanta International Airport                357862
Chicago OHare International Airport                             295062
Dallas Fort Worth International Airport                         246641
Denver International Airport                                    203903
Los Angeles International Airport                               197838
San Francisco International Airport                             152193
Phoenix Sky Harbor International Airport                        152127
George Bush Intercontinental Houston Airport                    151737
McCarran International Airport                                  137073
Minneapolis-St Paul International/Wold-Chamberlain Airport      117167
Detroit Metropolitan Wayne County Airport                       113985
General Edward Lawrence Logan International Airport             113285
Orlando International Airport                                   112262
La Guardia Airport                                              106337
Charlo

In [26]:
top_airports.isnull().sum()

YEAR                            0
MONTH                           0
DAY                             0
DAY_OF_WEEK                     0
AIRLINE                         0
FLIGHT_NUMBER                   0
TAIL_NUMBER                 11765
ORIGIN_AIRPORT                  0
DESTINATION_AIRPORT             0
SCHEDULED_DEPARTURE             0
DEPARTURE_TIME              55111
DEPARTURE_DELAY             55111
TAXI_OUT                    57078
WHEELS_OFF                  57078
SCHEDULED_TIME                  6
ELAPSED_TIME                67465
AIR_TIME                    67465
DISTANCE                        0
WHEELS_ON                   59414
TAXI_IN                     59414
SCHEDULED_ARRIVAL               0
ARRIVAL_TIME                59414
ARRIVAL_DELAY               67465
DIVERTED                        0
CANCELLED                       0
CANCELLATION_REASON             0
AIR_SYSTEM_DELAY                0
SECURITY_DELAY                  0
AIRLINE_DELAY                   0
LATE_AIRCRAFT_

In [27]:
#export top_airports csv
#top_airports.to_csv('top_airports.csv', encoding='utf-8', index=False)

In [28]:
#top_airports.to_json('top_airports.json', index=False)

In [49]:
#Remove columns that are not needed
trim_top_airports = top_airports.drop(['TOP30_x','TOP30_y','YEAR','FLIGHT_NUMBER','TAIL_NUMBER',"SCHEDULED_DEPARTURE",	"DEPARTURE_TIME",	"DEPARTURE_DELAY",	"TAXI_OUT",	"WHEELS_OFF",	"SCHEDULED_TIME",	"ELAPSED_TIME",	"AIR_TIME",	"DISTANCE",	"WHEELS_ON",	"TAXI_IN",	"SCHEDULED_ARRIVAL",	"ARRIVAL_TIME",	"ARRIVAL_DELAY",	"DIVERTED",	"CANCELLATION_REASON",
], axis=1)

In [51]:
trim_top_airports.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,...,WEATHER_DELAY,AIRLINE_NAME,ORIGIN_AIRPORT_NAME,ORIGIN_AIRPORT_CITY,ORIGIN_LAT,ORIGIN_LONG,DESTINATION_AIRPORT_NAME,DESTINATION_AIRPORT_CITY,DESTINATION_LAT,DESTINATION_LONG
0,1,1,THU,AA,LAX,PBI,Not Cancelled,0.0,0.0,0.0,...,0.0,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
1,1,2,FRI,AA,LAX,PBI,Not Cancelled,0.0,0.0,0.0,...,0.0,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
2,1,4,SUN,AA,LAX,PBI,Not Cancelled,0.0,0.0,0.0,...,0.0,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
3,1,5,MON,AA,LAX,PBI,Not Cancelled,23.0,0.0,0.0,...,0.0,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
4,1,8,THU,AA,LAX,PBI,Not Cancelled,16.0,0.0,35.0,...,0.0,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596


In [52]:
#Add "TOTAL_DELAY" column
trim_top_airports["TOTAL_DELAY"] = trim_top_airports['AIR_SYSTEM_DELAY'] + trim_top_airports['SECURITY_DELAY']+ trim_top_airports['AIRLINE_DELAY']+ trim_top_airports['LATE_AIRCRAFT_DELAY']+ trim_top_airports['WEATHER_DELAY']

In [53]:
#Function to recognize if the flight was delayed or not
def delayed (row):
    if row["TOTAL_DELAY"] > 0:
        return "Delayed"
    return "Not Delayed"


#trim_top_airports.apply(lambda row:delayed(row), axis=1)

#Add new column to identify if the flight was delayed or not

trim_top_airports['FLIGHT_DELAYED']=trim_top_airports.apply(lambda row:delayed(row), axis=1)

In [54]:
trim_top_airports.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,...,ORIGIN_AIRPORT_NAME,ORIGIN_AIRPORT_CITY,ORIGIN_LAT,ORIGIN_LONG,DESTINATION_AIRPORT_NAME,DESTINATION_AIRPORT_CITY,DESTINATION_LAT,DESTINATION_LONG,TOTAL_DELAY,FLIGHT_DELAYED
0,1,1,THU,AA,LAX,PBI,Not Cancelled,0.0,0.0,0.0,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,0.0,Not Delayed
1,1,2,FRI,AA,LAX,PBI,Not Cancelled,0.0,0.0,0.0,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,0.0,Not Delayed
2,1,4,SUN,AA,LAX,PBI,Not Cancelled,0.0,0.0,0.0,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,0.0,Not Delayed
3,1,5,MON,AA,LAX,PBI,Not Cancelled,23.0,0.0,0.0,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,23.0,Delayed
4,1,8,THU,AA,LAX,PBI,Not Cancelled,16.0,0.0,35.0,...,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,51.0,Delayed


In [55]:
columns = list(trim_top_airports.columns.values)
columns

['MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'AIRLINE',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'CANCELLED',
 'AIR_SYSTEM_DELAY',
 'SECURITY_DELAY',
 'AIRLINE_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'WEATHER_DELAY',
 'AIRLINE_NAME',
 'ORIGIN_AIRPORT_NAME',
 'ORIGIN_AIRPORT_CITY',
 'ORIGIN_LAT',
 'ORIGIN_LONG',
 'DESTINATION_AIRPORT_NAME',
 'DESTINATION_AIRPORT_CITY',
 'DESTINATION_LAT',
 'DESTINATION_LONG',
 'TOTAL_DELAY',
 'FLIGHT_DELAYED']

In [56]:
#Move "TOTAL_DELAY" column
trim_top_airports=trim_top_airports[['MONTH',	'DAY',	'DAY_OF_WEEK',	'AIRLINE',	'ORIGIN_AIRPORT',	'DESTINATION_AIRPORT',	'CANCELLED',	'AIR_SYSTEM_DELAY',	'SECURITY_DELAY',	'AIRLINE_DELAY',	'LATE_AIRCRAFT_DELAY',	'WEATHER_DELAY',	'TOTAL_DELAY','FLIGHT_DELAYED',	'AIRLINE_NAME',	'ORIGIN_AIRPORT_NAME',	'ORIGIN_AIRPORT_CITY',	'ORIGIN_LAT',	'ORIGIN_LONG',	'DESTINATION_AIRPORT_NAME',	'DESTINATION_AIRPORT_CITY',	'DESTINATION_LAT',	'DESTINATION_LONG',
]]
trim_top_airports.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,...,FLIGHT_DELAYED,AIRLINE_NAME,ORIGIN_AIRPORT_NAME,ORIGIN_AIRPORT_CITY,ORIGIN_LAT,ORIGIN_LONG,DESTINATION_AIRPORT_NAME,DESTINATION_AIRPORT_CITY,DESTINATION_LAT,DESTINATION_LONG
0,1,1,THU,AA,LAX,PBI,Not Cancelled,0.0,0.0,0.0,...,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
1,1,2,FRI,AA,LAX,PBI,Not Cancelled,0.0,0.0,0.0,...,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
2,1,4,SUN,AA,LAX,PBI,Not Cancelled,0.0,0.0,0.0,...,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
3,1,5,MON,AA,LAX,PBI,Not Cancelled,23.0,0.0,0.0,...,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
4,1,8,THU,AA,LAX,PBI,Not Cancelled,16.0,0.0,35.0,...,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596


In [57]:
trim_top_airports.isnull().sum()

MONTH                       0
DAY                         0
DAY_OF_WEEK                 0
AIRLINE                     0
ORIGIN_AIRPORT              0
DESTINATION_AIRPORT         0
CANCELLED                   0
AIR_SYSTEM_DELAY            0
SECURITY_DELAY              0
AIRLINE_DELAY               0
LATE_AIRCRAFT_DELAY         0
WEATHER_DELAY               0
TOTAL_DELAY                 0
FLIGHT_DELAYED              0
AIRLINE_NAME                0
ORIGIN_AIRPORT_NAME         0
ORIGIN_AIRPORT_CITY         0
ORIGIN_LAT                  0
ORIGIN_LONG                 0
DESTINATION_AIRPORT_NAME    0
DESTINATION_AIRPORT_CITY    0
DESTINATION_LAT             0
DESTINATION_LONG            0
dtype: int64

In [58]:
#export top_airports csv
#trim_top_airports.to_csv('trim_top_airports.csv', encoding='utf-8', index=False)

In [59]:
#Wrting to excel takes a very long time
#trim_top_airports.to_excel('trim_top_airports1.xlsx', encoding='utf-8', index=False)

In [60]:
trim_top_airports['MONTH'].unique()

array([ 1,  2,  3,  4,  5, 12,  6,  7,  8,  9, 11, 10], dtype=int64)

In [61]:
trim_top_airports.count()

MONTH                       3698598
DAY                         3698598
DAY_OF_WEEK                 3698598
AIRLINE                     3698598
ORIGIN_AIRPORT              3698598
DESTINATION_AIRPORT         3698598
CANCELLED                   3698598
AIR_SYSTEM_DELAY            3698598
SECURITY_DELAY              3698598
AIRLINE_DELAY               3698598
LATE_AIRCRAFT_DELAY         3698598
WEATHER_DELAY               3698598
TOTAL_DELAY                 3698598
FLIGHT_DELAYED              3698598
AIRLINE_NAME                3698598
ORIGIN_AIRPORT_NAME         3698598
ORIGIN_AIRPORT_CITY         3698598
ORIGIN_LAT                  3698598
ORIGIN_LONG                 3698598
DESTINATION_AIRPORT_NAME    3698598
DESTINATION_AIRPORT_CITY    3698598
DESTINATION_LAT             3698598
DESTINATION_LONG            3698598
dtype: int64

In [62]:
trim_top_airports.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3698598 entries, 0 to 5303583
Data columns (total 23 columns):
MONTH                       int64
DAY                         int64
DAY_OF_WEEK                 object
AIRLINE                     object
ORIGIN_AIRPORT              object
DESTINATION_AIRPORT         object
CANCELLED                   object
AIR_SYSTEM_DELAY            float64
SECURITY_DELAY              float64
AIRLINE_DELAY               float64
LATE_AIRCRAFT_DELAY         float64
WEATHER_DELAY               float64
TOTAL_DELAY                 float64
FLIGHT_DELAYED              object
AIRLINE_NAME                object
ORIGIN_AIRPORT_NAME         object
ORIGIN_AIRPORT_CITY         object
ORIGIN_LAT                  float64
ORIGIN_LONG                 float64
DESTINATION_AIRPORT_NAME    object
DESTINATION_AIRPORT_CITY    object
DESTINATION_LAT             float64
DESTINATION_LONG            float64
dtypes: float64(10), int64(2), object(11)
memory usage: 677.2+ MB


In [63]:
trim_top_airports['DAY_OF_WEEK'].unique()

array(['THU', 'FRI', 'SUN', 'MON', 'SAT', 'TUE', 'WED'], dtype=object)

In [64]:
#plt.bar(trim_top_airports["MONTH"],trim_top_airports["TOTAL_DELAY"])

#plt.show()

# Average Delay per weekday

In [65]:
df_delay_1 = trim_top_airports[(trim_top_airports['FLIGHT_DELAYED']=='Delayed') & (trim_top_airports['CANCELLED']=='Not Cancelled')]

In [66]:
df_delay_1

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,...,FLIGHT_DELAYED,AIRLINE_NAME,ORIGIN_AIRPORT_NAME,ORIGIN_AIRPORT_CITY,ORIGIN_LAT,ORIGIN_LONG,DESTINATION_AIRPORT_NAME,DESTINATION_AIRPORT_CITY,DESTINATION_LAT,DESTINATION_LONG
3,1,5,MON,AA,LAX,PBI,Not Cancelled,23.0,0.0,0.0,...,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
4,1,8,THU,AA,LAX,PBI,Not Cancelled,16.0,0.0,35.0,...,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
5,1,9,FRI,AA,LAX,PBI,Not Cancelled,19.0,0.0,0.0,...,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
11,1,18,SUN,AA,LAX,PBI,Not Cancelled,1.0,0.0,14.0,...,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
24,2,12,THU,AA,LAX,PBI,Not Cancelled,16.0,0.0,0.0,...,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
25,2,13,FRI,AA,LAX,PBI,Not Cancelled,16.0,0.0,0.0,...,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
31,2,22,SUN,AA,LAX,PBI,Not Cancelled,0.0,0.0,0.0,...,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
33,2,27,FRI,AA,LAX,PBI,Not Cancelled,0.0,0.0,119.0,...,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
34,2,28,SAT,AA,LAX,PBI,Not Cancelled,3.0,0.0,72.0,...,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
35,3,1,SUN,AA,LAX,PBI,Not Cancelled,0.0,0.0,93.0,...,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596


In [71]:
df_delay_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 725356 entries, 3 to 5303581
Data columns (total 23 columns):
MONTH                       725356 non-null int64
DAY                         725356 non-null int64
DAY_OF_WEEK                 725356 non-null object
AIRLINE                     725356 non-null object
ORIGIN_AIRPORT              725356 non-null object
DESTINATION_AIRPORT         725356 non-null object
CANCELLED                   725356 non-null object
AIR_SYSTEM_DELAY            725356 non-null float64
SECURITY_DELAY              725356 non-null float64
AIRLINE_DELAY               725356 non-null float64
LATE_AIRCRAFT_DELAY         725356 non-null float64
WEATHER_DELAY               725356 non-null float64
TOTAL_DELAY                 725356 non-null float64
FLIGHT_DELAYED              725356 non-null object
AIRLINE_NAME                725356 non-null object
ORIGIN_AIRPORT_NAME         725356 non-null object
ORIGIN_AIRPORT_CITY         725356 non-null object
ORIGIN_LAT       

In [69]:
#Remove columns that are not needed
trim_df_delay_1 = df_delay_1.drop(['AIRLINE','MONTH','DAY','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY'
], axis=1)
trim_df_delay_1.head()

,DAY_OF_WEEK,ORIGIN_AIRPORT,DESTINATION_AIRPORT,CANCELLED,TOTAL_DELAY,FLIGHT_DELAYED,AIRLINE_NAME,ORIGIN_AIRPORT_NAME,ORIGIN_AIRPORT_CITY,ORIGIN_LAT,ORIGIN_LONG,DESTINATION_AIRPORT_NAME,DESTINATION_AIRPORT_CITY,DESTINATION_LAT,DESTINATION_LONG
3,MON,LAX,PBI,Not Cancelled,23.0,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
4,THU,LAX,PBI,Not Cancelled,51.0,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
5,FRI,LAX,PBI,Not Cancelled,19.0,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
11,SUN,LAX,PBI,Not Cancelled,15.0,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
24,THU,LAX,PBI,Not Cancelled,16.0,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596


In [72]:
trim_df_delay_1.count()

DAY_OF_WEEK                 725356
ORIGIN_AIRPORT              725356
DESTINATION_AIRPORT         725356
CANCELLED                   725356
TOTAL_DELAY                 725356
FLIGHT_DELAYED              725356
AIRLINE_NAME                725356
ORIGIN_AIRPORT_NAME         725356
ORIGIN_AIRPORT_CITY         725356
ORIGIN_LAT                  725356
ORIGIN_LONG                 725356
DESTINATION_AIRPORT_NAME    725356
DESTINATION_AIRPORT_CITY    725356
DESTINATION_LAT             725356
DESTINATION_LONG            725356
dtype: int64

In [77]:
#export top_airports csv
trim_df_delay_1.to_csv('trim_df_delay_1.csv', encoding='utf-8', index=False)

In [75]:
grouped_trim_df_delay_1 = trim_df_delay_1.groupby(['DAY_OF_WEEK','ORIGIN_AIRPORT_NAME','ORIGIN_LAT','ORIGIN_LONG','DESTINATION_AIRPORT_NAME','DESTINATION_LAT','DESTINATION_LONG','DESTINATION_AIRPORT_CITY','TOTAL_DELAY'])['FLIGHT_DELAYED'].count().reset_index()

In [76]:
grouped_trim_df_delay_1.head()

,DAY_OF_WEEK,ORIGIN_AIRPORT_NAME,ORIGIN_LAT,ORIGIN_LONG,DESTINATION_AIRPORT_NAME,DESTINATION_LAT,DESTINATION_LONG,DESTINATION_AIRPORT_CITY,TOTAL_DELAY,FLIGHT_DELAYED
0,FRI,Baltimore/Washington International Thurgood Ma...,39.1754,-76.668297,Albany International Airport,42.748299,-73.801697,Albany,15.0,5
1,FRI,Baltimore/Washington International Thurgood Ma...,39.1754,-76.668297,Albany International Airport,42.748299,-73.801697,Albany,16.0,1
2,FRI,Baltimore/Washington International Thurgood Ma...,39.1754,-76.668297,Albany International Airport,42.748299,-73.801697,Albany,17.0,4
3,FRI,Baltimore/Washington International Thurgood Ma...,39.1754,-76.668297,Albany International Airport,42.748299,-73.801697,Albany,18.0,3
4,FRI,Baltimore/Washington International Thurgood Ma...,39.1754,-76.668297,Albany International Airport,42.748299,-73.801697,Albany,19.0,1


In [78]:
df_delay_2 = trim_top_airports[(trim_top_airports['CANCELLED']=='Not Cancelled')]

In [82]:
trim_df_delay_2 = df_delay_2.drop(['AIRLINE','MONTH','DAY','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','CANCELLED','LATE_AIRCRAFT_DELAY','WEATHER_DELAY'
], axis=1)
trim_df_delay_2.head()

,DAY_OF_WEEK,ORIGIN_AIRPORT,DESTINATION_AIRPORT,TOTAL_DELAY,FLIGHT_DELAYED,AIRLINE_NAME,ORIGIN_AIRPORT_NAME,ORIGIN_AIRPORT_CITY,ORIGIN_LAT,ORIGIN_LONG,DESTINATION_AIRPORT_NAME,DESTINATION_AIRPORT_CITY,DESTINATION_LAT,DESTINATION_LONG
0,THU,LAX,PBI,0.0,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
1,FRI,LAX,PBI,0.0,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
2,SUN,LAX,PBI,0.0,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
3,MON,LAX,PBI,23.0,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596
4,THU,LAX,PBI,51.0,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596


In [81]:
trim_df_delay_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3640888 entries, 0 to 5303582
Data columns (total 15 columns):
DAY_OF_WEEK                 object
ORIGIN_AIRPORT              object
DESTINATION_AIRPORT         object
CANCELLED                   object
TOTAL_DELAY                 float64
FLIGHT_DELAYED              object
AIRLINE_NAME                object
ORIGIN_AIRPORT_NAME         object
ORIGIN_AIRPORT_CITY         object
ORIGIN_LAT                  float64
ORIGIN_LONG                 float64
DESTINATION_AIRPORT_NAME    object
DESTINATION_AIRPORT_CITY    object
DESTINATION_LAT             float64
DESTINATION_LONG            float64
dtypes: float64(5), object(10)
memory usage: 444.4+ MB


In [89]:
origin_lookup_airport = ('origin_lookup_airport.csv')
destination_lookup_airport = ('destination_lookup_airport.csv')

In [90]:
origin_lookup_airport = pd.read_csv(origin_lookup_airport, low_memory=False)
origin_lookup_airport.head()

,ORIGIN_AIRPORT,ROUTE_ORIGIN
0,BTI,BTI
1,LUR,LUR
2,PIZ,PIZ
3,ITO,ITO
4,ORL,ORL


In [91]:
destination_lookup_airport = pd.read_csv(destination_lookup_airport, low_memory=False)
destination_lookup_airport.head()

,DESTINATION_AIRPORT,ROUTE_DESTINATION
0,BTI,BTI
1,LUR,LUR
2,PIZ,PIZ
3,ITO,ITO
4,ORL,ORL


In [92]:
trim_df_delay_merge_origin = pd.merge(trim_df_delay_2, origin_lookup_airport, on="ORIGIN_AIRPORT", how='inner')
trim_df_delay_merge_origin.head()

,DAY_OF_WEEK,ORIGIN_AIRPORT,DESTINATION_AIRPORT,TOTAL_DELAY,FLIGHT_DELAYED,AIRLINE_NAME,ORIGIN_AIRPORT_NAME,ORIGIN_AIRPORT_CITY,ORIGIN_LAT,ORIGIN_LONG,DESTINATION_AIRPORT_NAME,DESTINATION_AIRPORT_CITY,DESTINATION_LAT,DESTINATION_LONG,ROUTE_ORIGIN
0,THU,LAX,PBI,0.0,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX
1,FRI,LAX,PBI,0.0,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX
2,SUN,LAX,PBI,0.0,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX
3,MON,LAX,PBI,23.0,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX
4,THU,LAX,PBI,51.0,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX


In [93]:
trim_df_delay_merge_origin_destination = pd.merge(trim_df_delay_merge_origin, destination_lookup_airport, on="DESTINATION_AIRPORT", how='inner')
trim_df_delay_merge_origin_destination.head()

,DAY_OF_WEEK,ORIGIN_AIRPORT,DESTINATION_AIRPORT,TOTAL_DELAY,FLIGHT_DELAYED,AIRLINE_NAME,ORIGIN_AIRPORT_NAME,ORIGIN_AIRPORT_CITY,ORIGIN_LAT,ORIGIN_LONG,DESTINATION_AIRPORT_NAME,DESTINATION_AIRPORT_CITY,DESTINATION_LAT,DESTINATION_LONG,ROUTE_ORIGIN,ROUTE_DESTINATION
0,THU,LAX,PBI,0.0,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX,PBI
1,FRI,LAX,PBI,0.0,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX,PBI
2,SUN,LAX,PBI,0.0,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX,PBI
3,MON,LAX,PBI,23.0,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX,PBI
4,THU,LAX,PBI,51.0,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX,PBI


In [95]:
trim_df_delay_merge_origin_destination["route"]=trim_df_delay_merge_origin_destination['ROUTE_ORIGIN']+'-'+trim_df_delay_merge_origin_destination['ROUTE_DESTINATION']
trim_df_delay_merge_origin_destination.head(10)

,DAY_OF_WEEK,ORIGIN_AIRPORT,DESTINATION_AIRPORT,TOTAL_DELAY,FLIGHT_DELAYED,AIRLINE_NAME,ORIGIN_AIRPORT_NAME,ORIGIN_AIRPORT_CITY,ORIGIN_LAT,ORIGIN_LONG,DESTINATION_AIRPORT_NAME,DESTINATION_AIRPORT_CITY,DESTINATION_LAT,DESTINATION_LONG,ROUTE_ORIGIN,ROUTE_DESTINATION,route
0,THU,LAX,PBI,0.0,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX,PBI,LAX-PBI
1,FRI,LAX,PBI,0.0,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX,PBI,LAX-PBI
2,SUN,LAX,PBI,0.0,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX,PBI,LAX-PBI
3,MON,LAX,PBI,23.0,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX,PBI,LAX-PBI
4,THU,LAX,PBI,51.0,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX,PBI,LAX-PBI
5,FRI,LAX,PBI,19.0,Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX,PBI,LAX-PBI
6,SAT,LAX,PBI,0.0,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX,PBI,LAX-PBI
7,SUN,LAX,PBI,0.0,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX,PBI,LAX-PBI
8,THU,LAX,PBI,0.0,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX,PBI,LAX-PBI
9,FRI,LAX,PBI,0.0,Not Delayed,American Airlines,Los Angeles International Airport,Los Angeles,33.942501,-118.407997,Palm Beach International Airport,West Palm Beach,26.683201,-80.095596,LAX,PBI,LAX-PBI


In [105]:
#grouped_trim_df_delay_1 = trim_df_delay_1.groupby(['DAY_OF_WEEK','ORIGIN_AIRPORT_NAME','ORIGIN_LAT','ORIGIN_LONG','DESTINATION_AIRPORT_NAME','DESTINATION_LAT','DESTINATION_LONG','DESTINATION_AIRPORT_CITY','TOTAL_DELAY'])['FLIGHT_DELAYED'].count().reset_index()


route_delay_review=trim_df_delay_merge_origin_destination.groupby(['route','DAY_OF_WEEK','FLIGHT_DELAYED'])['ORIGIN_AIRPORT'].count().reset_index()

In [116]:
route_delay_review=trim_df_delay_merge_origin_destination.groupby(['route','DAY_OF_WEEK','FLIGHT_DELAYED']).agg({'ORIGIN_AIRPORT':'count','TOTAL_DELAY':'sum'}).reset_index()

In [125]:
route_delay_review=route_delay_review.rename(columns={"ORIGIN_AIRPORT": 'NUMBER_OF_FLIGHTS', "route":'ROUTE'})
route_delay_review.head(25)

,ROUTE,DAY_OF_WEEK,FLIGHT_DELAYED,NUMBER_OF_FLIGHTS,TOTAL_DELAY
0,ATL-ABE,FRI,Delayed,25,1257.0
1,ATL-ABE,FRI,Not Delayed,107,0.0
2,ATL-ABE,MON,Delayed,31,1797.0
3,ATL-ABE,MON,Not Delayed,108,0.0
4,ATL-ABE,SAT,Delayed,11,619.0
5,ATL-ABE,SAT,Not Delayed,79,0.0
6,ATL-ABE,SUN,Delayed,28,1607.0
7,ATL-ABE,SUN,Not Delayed,103,0.0
8,ATL-ABE,THU,Delayed,30,1933.0
9,ATL-ABE,THU,Not Delayed,106,0.0


In [127]:
route_delay_review['NUMBER_OF_FLIGHTS'].sum()

3640888

In [128]:
#export top_airports csv
route_delay_review.to_csv('route_delay_review.csv', encoding='utf-8', index=False)

In [130]:
delayed3routes = 'delayed3routes.csv'

#big file. includes the all the flights
delayed3routes = pd.read_csv(delayed3routes, low_memory=False)

delayed3routes.head()

,DAY_OF_WEEK,ROUTE,Delayed,Not Delayed
0,SUN,ORD-ATL,19%,81%
1,SUN,ORD-JFK,36%,64%
2,SUN,ORD-LAX,26%,74%
3,MON,ORD-ATL,26%,74%
4,MON,ORD-JFK,35%,65%


# Create Summary level DFs for D3 graphic

In [ ]:
count_of_flights = trim_top_airports[['ORIGIN_AIRPORT_NAME','ORIGIN_LAT','ORIGIN_LONG','DESTINATION_AIRPORT_NAME','DESTINATION_LAT','DESTINATION_LONG','DESTINATION_AIRPORT_CITY','CANCELLED']]

count_of_flights.head()

In [ ]:
grouped = count_of_flights.groupby(['ORIGIN_AIRPORT_NAME','ORIGIN_LAT','ORIGIN_LONG','DESTINATION_AIRPORT_NAME','DESTINATION_LAT','DESTINATION_LONG','DESTINATION_AIRPORT_CITY'])['CANCELLED'].count().reset_index()

In [ ]:
grouped.rename(columns={"CANCELLED":'count',"DESTINATION_AIRPORT_CITY":'city', "ORIGIN_AIRPORT_NAME":'origin',"DESTINATION_AIRPORT_NAME":'destination'}, inplace=True)

In [ ]:
grouped['country']='usa'
grouped.head()

In [ ]:
#export top_airports csv
grouped.to_csv('grouped.csv', encoding='utf-8', index=False)

In [ ]:
flights_airport_vega=grouped[['origin','destination', 'city','count']]

flights_airport_vega.head()
#export top_airports csv
flights_airport_vega_2015.to_csv('flights_airport_vega.csv', encoding='utf-8', index=False)

In [ ]:
airports_vega=grouped[['destination','city','country','DESTINATION_LAT','DESTINATION_LONG']]

airports_vega.head()

In [ ]:
#airports_vega.drop_duplicates(subset ="DESTINATION_AIRPORT_NAME", keep = False, inplace = True) 

In [ ]:
airports_vega_2015.to_csv('airports_vega.csv', encoding='utf-8', index=False)

# Summary DFs for Plotly and JS

In [ ]:
summary_of_flights = trim_top_airports[['ORIGIN_AIRPORT_NAME','ORIGIN_LAT','ORIGIN_LONG','DESTINATION_AIRPORT_NAME','DESTINATION_LAT','DESTINATION_LONG','DESTINATION_AIRPORT_CITY','CANCELLED','FLIGHT_DELAYED',"AIR_SYSTEM_DELAY",	"SECURITY_DELAY",	"AIRLINE_DELAY",	"LATE_AIRCRAFT_DELAY",	"WEATHER_DELAY",	"TOTAL_DELAY"
]]

summary_of_flights.head()

In [ ]:
grouped_summary_of_flights = summary_of_flights.groupby(['ORIGIN_AIRPORT_NAME','ORIGIN_LAT','ORIGIN_LONG','DESTINATION_AIRPORT_NAME','DESTINATION_LAT','DESTINATION_LONG','DESTINATION_AIRPORT_CITY'])["AIR_SYSTEM_DELAY","SECURITY_DELAY","AIRLINE_DELAY","LATE_AIRCRAFT_DELAY","WEATHER_DELAY","TOTAL_DELAY"
].sum().reset_index()

In [ ]:
grouped_summary_of_flights.head()

In [ ]:
grouped_summary_of_flights.to_csv('grouped_summary_of_flights.csv', encoding='utf-8', index=False)

In [ ]:
summary_of_flights_time = trim_top_airports[['MONTH','DAY','DAY_OF_WEEK','ORIGIN_AIRPORT_NAME','ORIGIN_LAT','ORIGIN_LONG','DESTINATION_AIRPORT_NAME','DESTINATION_LAT','DESTINATION_LONG','DESTINATION_AIRPORT_CITY','CANCELLED','FLIGHT_DELAYED',"AIR_SYSTEM_DELAY",	"SECURITY_DELAY",	"AIRLINE_DELAY",	"LATE_AIRCRAFT_DELAY",	"WEATHER_DELAY",	"TOTAL_DELAY"
]]

summary_of_flights_time.head()

In [ ]:
grouped_summary_of_flights_month = summary_of_flights_time.groupby(['MONTH','ORIGIN_AIRPORT_NAME','ORIGIN_LAT','ORIGIN_LONG','DESTINATION_AIRPORT_NAME','DESTINATION_LAT','DESTINATION_LONG','DESTINATION_AIRPORT_CITY'])["AIR_SYSTEM_DELAY","SECURITY_DELAY","AIRLINE_DELAY","LATE_AIRCRAFT_DELAY","WEATHER_DELAY","TOTAL_DELAY"
].sum().reset_index()

In [ ]:
grouped_summary_of_flights_month.head()

In [ ]:
grouped_summary_of_flights_month.to_csv('grouped_summary_of_flights_month.csv', encoding='utf-8', index=False)

In [ ]:
grouped_summary_of_flights_day_of_week = summary_of_flights_time.groupby(['DAY_OF_WEEK','ORIGIN_AIRPORT_NAME','ORIGIN_LAT','ORIGIN_LONG','DESTINATION_AIRPORT_NAME','DESTINATION_LAT','DESTINATION_LONG','DESTINATION_AIRPORT_CITY'])["AIR_SYSTEM_DELAY","SECURITY_DELAY","AIRLINE_DELAY","LATE_AIRCRAFT_DELAY","WEATHER_DELAY","TOTAL_DELAY"
].sum().reset_index()

In [ ]:
grouped_summary_of_flights_day_of_week.head()

In [ ]:
grouped_summary_of_flights_day_of_week.dtypes

In [ ]:
grouped_summary_of_flights_day_of_week.to_csv('grouped_summary_of_flights_day_of_week.csv', encoding='utf-8', index=False)

# 3 main routes

In [ ]:
#Import all needed datasets
routes = 'routes.csv'
ORDMIA = 'ORDMIA.csv'
ORDJFK ='ORDJFK.csv'
ORDLAX = 'ORDLAX.csv'



routes = pd.read_csv(routes, low_memory=False)
ORDMIA = pd.read_csv(ORDMIA, low_memory=False)
ORDJFK = pd.read_csv(ORDJFK, low_memory=False)
ORDLAX = pd.read_csv(ORDLAX, low_memory=False)

In [ ]:
ORDMIA.set_index('DAY', drop=True, append=False, inplace=True, verify_integrity=False)
ORDJFK.set_index('DAY', drop=True, append=False, inplace=True, verify_integrity=False)
ORDLAX.set_index('DAY', drop=True, append=False, inplace=True, verify_integrity=False)

In [ ]:
routes

In [ ]:
pivot=pd.pivot_table(routes, index=['ROUTE', 'DAY'])

pivot

In [ ]:
pivot.plot.bar()

In [ ]:
ORDLAX.plot.bar()

In [ ]:
ORDJFK.plot.bar()

In [ ]:
ORDLAX.plot.bar()

# MySQL - loading information to MySQL database

In [ ]:
#MySQL Connection

rds_connection_string = username + ":" + password +"@127.0.0.1/flights"

engine = create_engine(f'mysql://{rds_connection_string}')

In [ ]:
#Show available tables
engine.table_names()

In [ ]:
#Clear existing data in tables before loading information
#engine.execute("truncate table sample_flightsforsql")

In [ ]:
#Loads trim_top_airports dataframe into MySQL 
#trim_top_airports.to_sql(name='sample_flightsforsql', con=engine, if_exists='append', index=False)

# JSONs from Expedia

                                                        #ORD TO ATL

In [ ]:
ORD_ATL6_29 = 'https://raw.githubusercontent.com/alexkoynoff/JSON-files/master/ord-atl-flight-results%206-29.json'
# Load the first sheet of the JSON file into a data frame
ORD_ATL6_29 = pd.read_json(ORD_ATL6_29, orient='columns')
ORD_ATL6_29.insert(1, "date", '06/29/2019')

ORD_ATL6_29.head(3)

In [ ]:
ORD_ATL6_30 = 'https://raw.githubusercontent.com/alexkoynoff/JSON-files/master/ord-atl-flight-results%206-30.json'
# Load the first sheet of the JSON file into a data frame
ORD_ATL6_30 = pd.read_json(ORD_ATL6_30, orient='columns')
ORD_ATL6_30.insert(1, "date", '06/30/2019')

ORD_ATL6_30.head(3)

In [ ]:
ORD_ATL7_01 = 'https://raw.githubusercontent.com/alexkoynoff/JSON-files/master/ord-atl-flight-results%207-1.json'
# Load the first sheet of the JSON file into a data frame
ORD_ATL7_01 = pd.read_json(ORD_ATL7_01, orient='columns')
ORD_ATL7_01.insert(1, "date", '07/01/2019')

ORD_ATL7_01.head(3)

In [ ]:
ORD_ATL7_02 = 'https://raw.githubusercontent.com/alexkoynoff/JSON-files/master/ord-atl-flight-results%207-2.json'
# Load the first sheet of the JSON file into a data frame
ORD_ATL7_02 = pd.read_json(ORD_ATL7_02, orient='columns')
ORD_ATL7_02.insert(1, "date", '07/02/2019')

ORD_ATL7_02.head(3)

In [ ]:
ORD_ATL7_03 = 'https://raw.githubusercontent.com/alexkoynoff/JSON-files/master/ord-atl-flight-results%207-3.json'
# Load the first sheet of the JSON file into a data frame
ORD_ATL7_03 = pd.read_json(ORD_ATL7_03, orient='columns')
ORD_ATL7_03.insert(1, "date", '07/03/2019')

ORD_ATL7_03.head(3)

In [ ]:
ORD_ATL7_04 = 'https://raw.githubusercontent.com/alexkoynoff/JSON-files/master/ord-atl-flight-results%207-4.json'
# Load the first sheet of the JSON file into a data frame
ORD_ATL7_04 = pd.read_json(ORD_ATL7_04, orient='columns')
ORD_ATL7_04.insert(1, "date", '07/04/2019')

ORD_ATL7_04.head(3)

In [ ]:
ORD_ATL7_05 = 'https://raw.githubusercontent.com/alexkoynoff/JSON-files/master/ord-atl-flight-results%207-5.json'
# Load the first sheet of the JSON file into a data frame
ORD_ATL7_05 = pd.read_json(ORD_ATL7_05, orient='columns')
ORD_ATL7_05.insert(1, "date", '07/05/2019')

ORD_ATL7_05.head(3)

In [ ]:
ORD_ATL7_06 = 'https://raw.githubusercontent.com/alexkoynoff/JSON-files/master/ord-atl-flight-results%207-6.json'
# Load the first sheet of the JSON file into a data frame
ORD_ATL7_06 = pd.read_json(ORD_ATL7_06, orient='columns')
ORD_ATL7_06.insert(1, "date", '07/06/2019')

ORD_ATL7_06.head(3)

In [ ]:


json_concat_ORD_ATL = pd.concat([ORD_ATL6_29, ORD_ATL6_30,ORD_ATL7_01,ORD_ATL7_02,ORD_ATL7_03,ORD_ATL7_04,ORD_ATL7_05,ORD_ATL7_06],ignore_index=True)

json_concat_ORD_ATL.head(1000)

In [ ]:
json_concat_ORD_ATL.to_json('json_concat_ORD_ATL.json',orient='records')